In [1]:
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from numpy.linalg import inv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
%matplotlib widget

In [2]:
from sklearn import datasets
iris = datasets.load_iris()

In [3]:
type(iris)

sklearn.utils.Bunch

In [4]:
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [5]:
iris['target_names']

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [6]:
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

In [7]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   target             150 non-null    float64
dtypes: float64(5)
memory usage: 6.0 KB


In [9]:
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [10]:
g = sns.PairGrid(df)
g.map(sns.scatterplot)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
df_two_class = df[df['target'] > 0]
X = df_two_class[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
y = df_two_class['target']
X.shape, y.shape

((100, 4), (100,))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=34)

In [13]:
X_train.shape, y_train.shape

((70, 4), (70,))

In [14]:
class AdalineClassifier:
    def __init__(self):
        pass
    
    def fit(self, X_train, Y_train):
        self.w = np.matmul(inv(np.matmul(X_train.T, X_train)), np.matmul(X_train.T, Y_train))

    def predict(self, X_test):
        Y_pred = np.matmul(X_test, self.w)
        result = []
        for item in Y_pred:
            result.append(2 if item > 1.5 else 1)
        return np.array(result)
    
    def evaluate(self, y_pred, Y_test):
        acc = (len(np.where((y_pred - y_test)==0)[0])) / y_test.shape[0]
        return acc

In [15]:
model = AdalineClassifier()

In [16]:
model.fit(X_train, y_train)

/home/jhamed/miniconda3/envs/pyclass/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  


In [17]:
model.w

array([-0.10951976, -0.23406888,  0.33348548,  0.72585675])

In [18]:
y_pred = model.predict(X_test)
y_pred

array([1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2,
       1, 2, 2, 1, 1, 1, 2, 1])

In [19]:
model.evaluate(y_pred, y_test)

0.9666666666666667

# KNN

In [50]:
class KNearestNeighbors:
    def __init__(self, k):
        self.k = k
    
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        self.number_classes = 2
    
    def nearNeighbors(self, x_test):
        distance = np.sqrt(np.sum((x_test - self.X_train)**2, axis=1))
        near_neighbors = np.argsort(distance)[0:self.k]
        return near_neighbors
    
    def predict(self, x_test):
        near_neighbors = self.nearNeighbors(x_test)
        #print(near_neighbors)
        y = np.argmax(np.bincount(self.y_train[near_neighbors].astype('int64')))
        return y
    
    def evaluate(self, y_pred, y_test):
        score = np.sum(y_pred == np.array(y_test)) / len(y_pred)
        return score

In [51]:
knn = KNearestNeighbors(5)

In [52]:
knn.fit(X_train.values, y_train.values)

In [53]:
y_pred_knn = []
for item in X_test.values:
    y_pred_knn.append(knn.predict(item))
knn.evaluate(y_pred_knn, y_test)

0.9666666666666667